In [4]:
import os
import pandas as pd

def load_devtest_files(devtest_directory, specific_files):
    all_data = {}

    for file in specific_files:
        file_path = os.path.join(devtest_directory, file)
        
        if os.path.exists(file_path):
            try:
 
                with open(file_path, 'r', encoding='utf-8') as f:
                    sentences = f.readlines()
                sentences = [sentence.strip() for sentence in sentences if sentence.strip()]  # Remove empty lines and strip whitespace

                all_data[file] = sentences
                print(f"Loaded: {file} with {len(sentences)} sentences")
            except Exception as e:
                print(f"Error loading {file}: {e}")
        else:
            print(f"File not found: {file}")

    return all_data


devtest_directory = r'D:\Machine translation model\flores200_dataset\flores200_dataset\devtest'
specific_files = [
    'eng_Latn.devtest',
    'hin_Deva.devtest',
    'mar_Deva.devtest',
    'guj_Gujr.devtest',
    'ben_Beng.devtest',
    'tel_Telu.devtest',
    'tam_Taml.devtest',
]


loaded_data = load_devtest_files(devtest_directory, specific_files)

Loaded: eng_Latn.devtest with 1012 sentences
Loaded: hin_Deva.devtest with 1012 sentences
Loaded: mar_Deva.devtest with 1012 sentences
Loaded: guj_Gujr.devtest with 1012 sentences
Loaded: ben_Beng.devtest with 1012 sentences
Loaded: tel_Telu.devtest with 1012 sentences
Loaded: tam_Taml.devtest with 1012 sentences


In [5]:
def save_random_samples_to_excel(loaded_data, sample_size=150):
    english_sentences = loaded_data['eng_Latn.devtest']
    
    output_directory = r'D:\Machine translation model\Machine Translation Model\OUTPUT MT'

    sample_size = min(sample_size, len(english_sentences))

    for lang_file, lang_sentences in loaded_data.items():
        if lang_file != 'eng_Latn.devtest':
    
            min_len = min(len(english_sentences), len(lang_sentences))
            if min_len < sample_size:
                print(f"Not enough sentences to sample for {lang_file}. Available: {min_len}")
                continue
            
            english_sample = pd.Series(english_sentences).sample(n=sample_size, random_state=42).reset_index(drop=True)
            lang_sample = pd.Series(lang_sentences).sample(n=sample_size, random_state=42).reset_index(drop=True)

            parallel_df = pd.DataFrame({
                'English': english_sample,
                'Translation': lang_sample
            })

            lang_code = lang_file.split('_')[0]
            excel_filename = os.path.join(output_directory, f'parallel_dataset_eng_{lang_code}.xlsx')
            parallel_df.to_excel(excel_filename, index=False)
            print(f'Saved: {excel_filename} with {len(parallel_df)} entries.')
save_random_samples_to_excel(loaded_data)


Saved: D:\Machine translation model\Machine Translation Model\OUTPUT MT\parallel_dataset_eng_hin.xlsx with 150 entries.
Saved: D:\Machine translation model\Machine Translation Model\OUTPUT MT\parallel_dataset_eng_mar.xlsx with 150 entries.
Saved: D:\Machine translation model\Machine Translation Model\OUTPUT MT\parallel_dataset_eng_guj.xlsx with 150 entries.
Saved: D:\Machine translation model\Machine Translation Model\OUTPUT MT\parallel_dataset_eng_ben.xlsx with 150 entries.
Saved: D:\Machine translation model\Machine Translation Model\OUTPUT MT\parallel_dataset_eng_tel.xlsx with 150 entries.
Saved: D:\Machine translation model\Machine Translation Model\OUTPUT MT\parallel_dataset_eng_tam.xlsx with 150 entries.


In [9]:
import pandas as pd
import os

def load_parallel_samples(output_directory, languages):
    sampled_data = {}
    for lang_code in languages:
        excel_filename = os.path.join(output_directory, f'parallel_dataset_eng_{lang_code}.xlsx')
        if os.path.exists(excel_filename):
            df = pd.read_excel(excel_filename)
            sampled_data[lang_code] = df
            print(f"Loaded: {excel_filename} with {len(df)} entries.")
        else:
            print(f"File not found: {excel_filename}")
    return sampled_data

output_directory = r'D:\Machine translation model\Machine Translation Model\OUTPUT MT'
languages = ['hin', 'mar', 'guj', 'ben', 'tel', 'tam'] 

sampled_data = load_parallel_samples(output_directory, languages)

Loaded: D:\Machine translation model\Machine Translation Model\OUTPUT MT\parallel_dataset_eng_hin.xlsx with 150 entries.
Loaded: D:\Machine translation model\Machine Translation Model\OUTPUT MT\parallel_dataset_eng_mar.xlsx with 150 entries.
Loaded: D:\Machine translation model\Machine Translation Model\OUTPUT MT\parallel_dataset_eng_guj.xlsx with 150 entries.
Loaded: D:\Machine translation model\Machine Translation Model\OUTPUT MT\parallel_dataset_eng_ben.xlsx with 150 entries.
Loaded: D:\Machine translation model\Machine Translation Model\OUTPUT MT\parallel_dataset_eng_tel.xlsx with 150 entries.
Loaded: D:\Machine translation model\Machine Translation Model\OUTPUT MT\parallel_dataset_eng_tam.xlsx with 150 entries.


In [10]:
import openai
from nltk.translate.bleu_score import sentence_bleu


openai.api_key = 'OpenAi api key'

def evaluate_translation(english_sentence, actual_translation, target_language):
    prompt = f"Translate the following sentence to {target_language}:\n\n{english_sentence}\n\nTranslation:"
    
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",  # Use GPT-4 Mini here
            messages=[{"role": "user", "content": prompt}],
            max_tokens=60
        )
        gpt_translation = response['choices'][0]['message']['content'].strip()
        bleu_score = sentence_bleu([actual_translation.split()], gpt_translation.split())
        return bleu_score, gpt_translation
    
    except Exception as e:
        print(f"Error evaluating translation: {e}")
        return None, None

def evaluate_samples_and_save_to_excel(sampled_data, output_directory):
    for lang_code, data in sampled_data.items():
        target_language = lang_code.split('_')[0]  # Extract language code
        english_sentences = data['English']
        actual_translations = data['Translation']

        bleu_scores = []
        gpt_translations = []

        for english_sentence, actual_translation in zip(english_sentences, actual_translations):
            bleu_score, gpt_translation = evaluate_translation(english_sentence, actual_translation, target_language)
            bleu_scores.append(bleu_score)
            gpt_translations.append(gpt_translation)

            print(f"Evaluated: {english_sentence} -> {gpt_translation} | BLEU Score: {bleu_score}")

        evaluation_df = pd.DataFrame({
            'English': english_sentences,
            'Actual_Translation': actual_translations,
            'GPT_Translation': gpt_translations,
            'BLEU_Score': bleu_scores
        })

        excel_filename = os.path.join(output_directory, f'evaluation_eng_{lang_code}.xlsx')
        evaluation_df.to_excel(excel_filename, index=False)
        print(f"Saved evaluation results to: {excel_filename}")

evaluate_samples_and_save_to_excel(sampled_data, output_directory)


Evaluated: Vatican City's population is around 800. It is the smallest independent country in the world and the country with the lowest population. -> वेटिकन सिटी की जनसंख्या लगभग 800 है। यह दुनिया का सबसे छोटा स्वतंत्र देश है और यह सबसे कम जनसंख्या वाला देश है। | BLEU Score: 0.3839817133079349


C:\Drivers\Anaconda3\envs\virenve2\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Evaluated: All citizens of Vatican City are Roman Catholic. -> सभी वेटिकन सिटी के नागरिक रोमन कैथोलिक हैं। | BLEU Score: 7.803659209895733e-78
Evaluated: It has a notably wide variety of plant communities, due to its range of microclimates, differing soils and varying levels of altitude. -> इसमें पौधों के समुदायों की विशेष रूप से विस्तृत विविधता है, जो इसके विभिन्न सूक्ष्म जलवायु, भिन्न मिट्टियों और ऊँचाई के विभिन्न स्तरों के कारण है। | BLEU Score: 0.3321678143250398
Evaluated: The Amazon River is the second longest and the biggest river on Earth. It carries more than 8 times as much water as the second biggest river. -> अमेज़न नदी पृथ्वी पर दूसरी सबसे लंबी और सबसे बड़ी नदी है। यह दूसरी सबसे बड़ी नदी से 8 गुना अधिक पानी लेकर चलती है। | BLEU Score: 0.49124158433111575
Evaluated: Lion prides act much like packs of wolves or dogs, animals surprisingly similar to lions (but not other big cats) in behavior, and also very deadly to their prey. -> सिंहों के झुंड पूरी तरह से भेड़ियों या कुत्तो

C:\Drivers\Anaconda3\envs\virenve2\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Evaluated: Sadly, as newer methods of writing have emerged, the pencil has been relegated to lesser status and uses. -> दुर्भाग्यवश, जैसे-जैसे नए लेखन के तरीके सामने आए हैं, पेंसिल को कम महत्वपूर्ण स्थिति और उपयोगों में relegated कर दिया गया है। | BLEU Score: 4.778646615255477e-155
Evaluated: At the same time, the German navy, using mainly U-boats, was trying to stop this traffic. -> एक ही समय में, जर्मन नौसेना, मुख्य रूप से यू-बोट का उपयोग करते हुए, इस यातायात को रोकने की कोशिश कर रही थी। | BLEU Score: 0.43600387912116445
Evaluated: Using ships to transport goods is by far the most efficient way to move large amounts of people and goods across oceans. -> जहाजों का उपयोग करके सामान का परिवहन करना महासागरों के पार बड़ी मात्रा में लोगों और सामान को ले जाने का सबसे कुशल तरीका है। | BLEU Score: 0.21773420834233131
Evaluated: However, the percentage of XDR-TB in the entire group of people with tuberculosis still seems to be low; 6,000 of the total 330,000 people infected at any particular m

C:\Drivers\Anaconda3\envs\virenve2\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Evaluated: He referred to the rumors as "political chatter and silliness". -> उसने अफवाहों का उल्लेख "राजनीतिक चिचियों और बेतुकापन" के रूप में किया। | BLEU Score: 1.1640469867513693e-231
Evaluated: Since the foundation of Asunción in 1537, Paraguay has managed to keep a lot of its indigenous character and identity. -> 1537 में आसुंसियन की स्थापना के बाद से, पाराग्वे ने अपनी कई स्थानीय विशेषताओं और पहचान को बनाए रखने में सफलता हासिल की है। | BLEU Score: 0.37230919026016435
Evaluated: First among its 78 recommendations is that a new diplomatic initiative should be taken before the end of this year to secure Iraq’s borders against hostile interventions and to re-establish diplomatic relations with its neighbors. -> इसके 78 सिफारिशों में से पहली यह है कि इस वर्ष के अंत से पहले एक नई कूटनीतिक पहल की जानी चाहिए ताकि इराक की सीमाओं को दुश्मन के हस्तक्षेप के खिलाफ सुरक्षित किया जा सके और अपने पड़ोसियों के साथ क | BLEU Score: 0.20998068783040205
Evaluated: Two songs from the movie, Audition (Th